Modelo LSTM de Google Colab

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import os

# Ruta del nuevo archivo CSV
csv = '/content/drive/MyDrive/archive/appliance_usage_data2.csv'

# Ajustar la longitud de la secuencia
sequence_length = 10

# Hiperparámetros de la red LSTM
num_features = 1
epochs = 10

# Verificar la disponibilidad de GPU
if tf.test.gpu_device_name():
    print('GPU encontrada:', tf.test.gpu_device_name())
else:
    print('No se encontró GPU. Asegúrate de habilitar la GPU en "Entorno de ejecución".')

# Comprobar si el archivo del modelo existe
model_filename = 'modelo.h5'
if os.path.exists(model_filename):
    print("Cargando modelo existente")
    # Si el archivo del modelo existe, cargar el modelo
    model = tf.keras.models.load_model(model_filename)
else:
    # Si el archivo del modelo no existe, crear un nuevo modelo
    with tf.device('/device:GPU:0'):
        model = Sequential()
        model.add(LSTM(50, input_shape=(sequence_length, num_features)))
        model.add(Dense(num_features))
        model.compile(optimizer='adam', loss='mse')

# Crear un DataFrame vacío para almacenar los datos
df = pd.read_csv(csv)

# Convertir la columna de timestamp en datetime
df['Fecha'] = pd.to_datetime(df['Fecha'] + ' ' + df['Hora'].astype(str) + ':00:00')

# Establecer la columna de timestamp como el índice del dataframe
df = df.set_index('Fecha')

# Normalizar los datos de potencia
scaler = StandardScaler()
df['Watts usados'] = scaler.fit_transform(df['Watts usados'].values.reshape(-1, 1))

with tf.device('/device:GPU:0'):
    # Crear secuencias de datos para entrenar la red LSTM
    data = []
    for i in range(len(df) - sequence_length):
        data.append(df['Watts usados'].iloc[i:i+sequence_length])

    # Convertir la lista de secuencias en un array numpy
    data = np.array(data)

    # Asegúrate de que la cantidad de datos de entrenamiento sea un múltiplo de sequence_length
    train_size = len(data) - (len(data) % sequence_length)
    x_train = data[:train_size, :-1]
    y_train = data[:train_size, -1]
    x_train = x_train.reshape(-1, sequence_length, num_features)

    # Asegúrate de que x_train y y_train tengan la misma longitud
    min_length = min(len(x_train), len(y_train))
    x_train = x_train[:min_length]
    y_train = y_train[:min_length]

    # Entrenar el modelo
    model.fit(x_train, y_train, epochs=epochs)

    # Guardar el modelo entrenado
    model.save('modelo.h5')

# Cargar el modelo en el futuro
modelo_cargado = tf.keras.models.load_model('modelo.h5')


Modelo con clasificación binaria

In [125]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pickle

# Cargar los datos desde el archivo CSV
data = pd.read_csv('appliance_usage_data2.csv')

# Normalizar los datos
scaler = MinMaxScaler()
data['Watts usados'] = scaler.fit_transform(data['Watts usados'].values.reshape(-1, 1))

# Dividir los datos en conjuntos de entrenamiento y prueba
X = data[['Hora', 'Probabilidad de fallo', 'Porcentaje de fallos']].values
y = data['Reporte de fallo'].values  # Clasificación binaria basada en el Reporte de fallo

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de clasificación binaria
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluar el modelo
accuracy = model.evaluate(X_test, y_test)[1]
print(f'Accuracy: {accuracy}')

# Hacer predicciones
y_pred = model.predict(X_test)

# Predicción de si el electrodoméstico está fallando o no
print('Predicciones:', y_pred[:5])
print('Valores reales:', y_test[:5])

# Guardar el modelo en un archivo
with open('modelo_clasificacion_fallo.pkl', 'wb') as file:
    pickle.dump(model, file)

Epoch 1/10
12500/12500 [==============================] - 23s 2ms/step - loss: 0.0095 - accuracy: 0.9977
Epoch 2/10
12500/12500 [==============================] - 22s 2ms/step - loss: 1.2541e-04 - accuracy: 1.0000
Epoch 3/10
12500/12500 [==============================] - 21s 2ms/step - loss: 8.2202e-05 - accuracy: 1.0000
Epoch 4/10
12500/12500 [==============================] - 22s 2ms/step - loss: 6.9933e-05 - accuracy: 1.0000
Epoch 5/10
12500/12500 [==============================] - 23s 2ms/step - loss: 3.6440e-05 - accuracy: 1.0000
Epoch 6/10
12500/12500 [==============================] - 24s 2ms/step - loss: 5.1890e-05 - accuracy: 1.0000
Epoch 7/10
12500/12500 [==============================] - 21s 2ms/step - loss: 3.7058e-05 - accuracy: 1.0000
Epoch 8/10
12500/12500 [==============================] - 21s 2ms/step - loss: 3.5266e-05 - accuracy: 1.0000
Epoch 9/10
12500/12500 [==============================] - 21s 2ms/step - loss: 3.5418e-05 - accuracy: 1.0000
Epoch 10/10
3125/3125 [

In [8]:
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler

# Cargar el modelo desde el archivo
with open('modelo_clasificacion_fallo.pkl', 'rb') as file:
    model = pickle.load(file)

# Datos de entrada
# Abriendo el archivo CSV
data = pd.read_csv('nuevos_datos.csv')

# Normalizar los datos
scaler = MinMaxScaler()
data['Watts usados'] = scaler.fit_transform(data['Watts usados'].values.reshape(-1, 1))

# Preparar los datos para la predicción
X_pred = data[['Hora', 'Probabilidad de fallo', 'Porcentaje de fallos']].values

# Hacer la predicción
y_pred = model.predict(X_pred)

print('Predicciones:', y_pred)

1/1 [==============================] - 0s 128ms/step
Predicciones: [[4.3968379e-28]
 [1.0000000e+00]
 [7.0673190e-10]
 [1.0000000e+00]
 [1.0000000e+00]
 [1.0000000e+00]
 [4.6599024e-07]
 [1.9746300e-09]
 [2.1188448e-20]
 [1.0004136e-12]
 [1.2798066e-13]]


Probado de otra forma el modelo de clasificación binaria.

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Cargar los datos desde el archivo CSV
data = pd.read_csv('appliance_usage_data2.csv')

# Normalizar los datos directamente en el DataFrame
data['Watts usados'] = (data['Watts usados'] - data['Watts usados'].min()) / (data['Watts usados'].max() - data['Watts usados'].min())

# Dividir los datos en conjuntos de entrenamiento y prueba
X = data[['Hora', 'Probabilidad de fallo', 'Porcentaje de fallos']].values
y = data['Watts usados'].values  # Valores normalizados entre 0 y 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión (ya que ahora y es una variable continua)
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(1, activation='linear'))  # Linear activation for regression
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluar el modelo
mse, mae = model.evaluate(X_test, y_test)
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')

# Hacer predicciones
y_pred = model.predict(X_test)

# Comparando los valores reales con las predicciones.
print('Valores reales:', y_test[:5])
print('Predicciones:', y_pred[:5])

# Guardando las predicciones.
np.savetxt('predicciones_binarias_normalizadas.txt', y_pred)

# Guardar el modelo en un archivo
with open('modelo_regresion.pkl', 'wb') as file:
    pickle.dump(model, file)